In [13]:
def calibrate_vp():
  
   keys=['a_h_m','a_h_ratio_2','p_deltae','a_h_base','p_nw','a_b_m','e_tau_S','p_nr','e_rb','a_l_m','a_l_scale']
   a_h_m,a_h_ratio_2,p_deltae,a_h_base,p_nw,a_b_m,e_tau_S,p_nr,e_rb,a_l_m,a_l_scale=[pars.get(key)[0] for key in keys]
   
   
   x0=.3
   a_h_var_phi6, info, ier, mesg = fsolve(calibrate_vp_human_c,x0,full_output=True)
   if ier!=1:
       sys.exit(mesg)
   
    #define parameters    
   a_h_var_phi7=a_h_var_phi6*a_h_m**(a_h_var_phi6-1)
   a_h_var_phi5=(a_h_ratio_2-(1-p_deltae)**a_h_base)/(a_h_base**a_h_var_phi6-a_h_var_phi7*a_h_base)
    
    #vector of a=0 to 43P
   ht = np.arange(0, p_nw, 1)
    #vector of x(a)/h0
   s_x_h0=a_h_var_phi5*(ht**a_h_var_phi6-a_h_var_phi7*ht)
    #vector of ha/h0
   s_h_h0=(1-p_deltae)**ht+s_x_h0
    
    #DISCOUNT RATE
    #finds parameter of discount rate (beta) function
   x0=.5 #start ng guess
    
   a_b_var_phi1, info, ier, mesg = fsolve(calibrate_vp_beta,x0,full_output=True)
   if ier!=1:
       sys.exit(mesg)
  
    #fdefine parameters      
   a_b_var_phi2=a_b_var_phi1*(a_b_m+1)**(a_b_var_phi1-1)
    
    #fvector of a=0 to 61
   
   bt = np.arange(0, p_nw+p_nr, 1)
       #fvector of beta*R**a
   p_b=(bt+1)**a_b_var_phi1-a_b_var_phi2*bt
    #fvector of beta*R**a
   p_b=p_b/(1+e_rb*(1-e_tau_S))**(bt)
    #fnormalize by initial value
   p_b=p_b/p_b[0]
      
  
    #fLABOR HOURS
    #ffinds parameter of discount rate (beta) function
    
   a_l_var_phi3, info, ier, mesg = fsolve(calibrate_vp_labor,x0,full_output=True)
   if ier!=1:
      sys.exit(mesg)
   a_l_var_phi4=a_l_var_phi3*(a_l_m+1)**(a_l_var_phi3-1)
    #fvector of a=0 to 43
   
   lt=np.arange(0, p_nw, 1)
    #fadd 1 to this since a+1 is in the calculations
   lt=lt+1
    #fvector of l values-not normalized
   e_l=lt**a_l_var_phi3-a_l_var_phi4*lt
    #fector of l values-not normalized
    #fe_l=e_l/mean(e_l)*a_l_scale
   e_l=e_l/np.mean(e_l)*a_l_scale
   e_l=e_l/np.mean(e_l)*a_l_scale
   pars['e_l']=[e_l,'the series of labor hours']
   pars['s_h_h0']=[s_h_h0,'???']
   pars['p_b']=[p_b,'???']
   pars['s_x_h0']=[s_x_h0,'???']
   pars['p_b']=[p_b,'???']
  

 

